In [1]:
import os

In [2]:
!pip install -U sentence-transformers

In [3]:
!pip install bio

  Using cached bio-1.3.9-py3-none-any.whl (270 kB)
     |████████████████████████████████| 16.7 MB 319 kB/s eta 0:00:01     |███████████████████████████████▊| 16.6 MB 405 kB/s eta 0:00:01
  Using cached mygene-3.2.2-py2.py3-none-any.whl (5.4 kB)
  Created wheel for biopython: filename=biopython-1.79-cp310-cp310-macosx_10_9_x86_64.whl size=2268328 sha256=77ec702d9e7421dfd70efa458c2ed0dbfefa593f9b72226e92792c14609042fa
  Stored in directory: /Users/eduardfartushnyy/Library/Caches/pip/wheels/7c/46/dd/3e2f23acbbe0dd5b3744d069ad905e0218a29c879291beeef5
Successfully built biopython


In [ ]:
import pandas as pd
import time
from tqdm import tqdm
import seaborn as sns
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

/Users/eduardfartushnyy/opt/anaconda3/envs/scientificProject/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from .autonotebook import tqdm as notebook_tqdm

ImportError: attempted relative import with no known parent package

In [2]:
from Bio import Entrez
from Bio import Medline

def clear_string(inp_str):
    inp_str = inp_str.replace('"', ' ')
    inp_str = inp_str.replace("'", " ")
    inp_str = inp_str.replace("[", " ")
    inp_str = inp_str.replace("]", " ")
    inp_str = inp_str.replace('\\', " ")
    inp_str = inp_str.replace('/', " ")

    return inp_str

# search_strings = ["nafld and 2020/01/01:2022/02/08[dp]",
#                   "nafld and 2017/01/01:2019/12/31[dp]",
#                   "nafld and 2010/01/01:2016/12/31[dp]",
#                   "nafld and 2000/01/01:2009/12/31[dp]"]

# search_strings = ["nafld and 2020/05/01:2022/06/15[dp]",
#                   "nafld and 2017/01/01:2020/04/30[dp]"]
                  
search_strings = ["nafld and 2021/01/01:2022/06/15[dp]"]
                  
# search_strings = ["nafld and 2020/01/01:2021/10/26[dp]"]

# search_strings = ["glioblastoma AND surgery and 1994/01/01:2021/11/04[dp]"]

# search_strings = ["acute limb ischemia"]

# search_strings = ["remdesivir"]


Entrez.email = "e.p@d_health.pro"  # Говорю NCBI кто я 
docs = []  # готовлю список для документов
i=0        # инициализирую счетчик документов  

d = {'title': [], 'abstract': [], 'mesh':[]}
df = pd.DataFrame(data=d)
print('Original DataFrame\n------------------') 
print(df) 


for search_str in search_strings:
  handle = Entrez.esearch(db="pubmed", sort='relevance', term=search_str,  retmax='10000')
  record = Entrez.read(handle)
  rec_count = record['Count']
  print(search_str,' - ' ,rec_count)

  idlist = record["IdList"]
  handle = Entrez.efetch(db="pubmed", id=idlist, rettype="medline", retmode="text")
  records = Medline.parse(handle)

  for record in records:
    # названия
    try: 
      title = clear_string(str(record['TI']))
    except:
      title = ''
    # абстракты  
    try:
      abstract = clear_string(str(record['AB']))
    except:
      abstract = 'no text'  
    # mesh-термины
    try:
      mesh = clear_string(str(record['MH']))
    except:
      mesh = ' '  

    new_row = {'title':title, 'abstract':abstract, 'mesh':mesh} #append row to the dataframe 
    df = df.append(new_row, ignore_index=True)

    # ap = title + abstract + mesh
    # docs.append(ap)
    i = i+1
    if i % 1000 == 0:
      print(i,df.iloc[[i-1]])

# печатаем последний загруженный документ


NameError: name 'pd' is not defined

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
from os import path
from wordcloud import WordCloud

# get data directory (using getcwd() is needed to support running example in generated IPython notebook)
d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()

# какая выборка документов
d1=0 #@param {type: "integer"}
d2=120 #@param {type: "integer"}
text =''
for i in range(d1, d2):
  text = text + df['title'].iloc[i]
  text = text + df['mesh'].iloc[i]
  text = text + df['abstract'].iloc[i]

print(text)
# Generate a word cloud image

wordcloud = WordCloud(max_font_size=36, max_words=100, background_color="white").generate(text)

# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

# lower max_font_size
# max_font_size=32   #@param {type: "integer"}
font_size =  20#@param {type:"integer"}
wordcloud = WordCloud(max_font_size=font_size, background_color="white").generate(text)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# The pil way (if you don't have matplotlib)
# image = wordcloud.to_image()
# image.show()

In [ ]:
# data = pd.read_csv('../input/wikipedia-movie-plots/wiki_movie_plots_deduped.csv',memory_map=True)
df.info()

The dataset contains publication by NAFLD topic  ( 19,600 articles) from NCBI PubMed storage. 
Column descriptions are listed below:

Title — Article title
Mesh — Article mesh terms
Abstract — Article abstract text

In [ ]:
import gc
gc.collect()

26202

In [ ]:
df.dropna(inplace=True)
df.drop_duplicates(subset=['abstract'],inplace=True)

In [ ]:
df['doc_len'] = df['abstract'].apply(lambda words: len(words.split()))
max_seq_len = np.round(df['doc_len'].mean() + df['doc_len'].std()).astype(int)
sns.distplot(df['doc_len'], hist=True, kde=True, color='g', label='doc len')
plt.axvline(x=max_seq_len, color='g', linestyle='--', label='max len')
plt.title('plot length'); plt.legend()
plt.show()

In [17]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import faiss
encoded_data = model.encode(df.abstract.tolist(), batch_size=40, show_progress_bar=True)

Batches:   0%|          | 0/132 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [50]:
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(df))))
faiss.write_index(index, 'title.index')

In [52]:
def fetch_movie_info(dataframe_idx):
    info = df.iloc[dataframe_idx]
    meta_dict = {}
    meta_dict['title'] = info['title']
    return meta_dict
    
def search(query, top_k, index, model):
    t=time.time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    print('>>>> Results in Total Time: {}'.format(time.time()-t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_movie_info(idx) for idx in top_k_ids]
    return results

In [53]:
from pprint import pprint

query="acute NAFLD desiase"
results=search(query, top_k=5, index=index, model=model)

print("\n")
for result in results:
    print('\t',result)

>>>> Results in Total Time: 0.07195544242858887


	 {'title': 'Involvement of Ceramides in Non-Alcoholic Fatty Liver Disease (NAFLD) Atherosclerosis (ATS) Development: Mechanisms and Therapeutic Targets.'}
	 {'title': 'Acute Elevated Resistin Exacerbates Mitochondrial Damage and Aggravates Liver Steatosis Through AMPK PGC-1alpha Signaling Pathway in Male NAFLD Mice.'}
	 {'title': 'Association between the soluble receptor for advanced glycation end products (sRAGE) and NAFLD in participants in the Atherosclerosis Risk in Communities Study.'}
	 {'title': 'Intrahepatic cholestasis of pregnancy: An under recognised complication of maternal NAFLD?'}
	 {'title': 'Myosteatosis in NAFLD patients correlates with plasma Cathepsin D.'}


In [54]:
from pprint import pprint

query="advanced glycation end products"
results=search(query, top_k=5, index=index, model=model)

print("\n")
for result in results:
    print('\t',(result))

>>>> Results in Total Time: 0.07196450233459473


	 {'title': 'Liraglutide inhibits receptor for advanced glycation end products (RAGE) reduced form of nicotinamide-adenine dinucleotide phosphate (NAPDH) signaling to ameliorate non-alcoholic fatty liver disease (NAFLD) in vivo and vitro.'}
	 {'title': 'Advanced glycation end product: A potential biomarker for risk stratification of non-alcoholic fatty liver disease in ELSA-Brasil study.'}
	 {'title': 'Association between the soluble receptor for advanced glycation end products (sRAGE) and NAFLD in participants in the Atherosclerosis Risk in Communities Study.'}
	 {'title': 'Advanced Glycation End-Products in Common Non-Infectious Liver Diseases: Systematic Review and Meta-Analysis.'}
	 {'title': 'Accumulation of Toxic Advanced Glycation End-Products Induces Cytotoxicity and Inflammation in Hepatocyte-Like Cells Differentiated from Human Induced Pluripotent Stem Cells.'}


In [56]:
paragraphs=df.title.tolist()

In [57]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
tokenizer = T5Tokenizer.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
model = T5ForConditionalGeneration.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
model.eval()

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (

In [58]:
#Select the device
device = 'cuda'
model.to(device)

RuntimeError: ignored

In [59]:
# Parameters for generation
batch_size = 16 #Batch size
num_queries = 5 #Number of queries to generate for every paragraph
max_length_paragraph = 512 #Max length for paragraph
max_length_query = 64   #Max length for output query

In [60]:
def _removeNonAscii(s): return "".join(i for i in s if ord(i) < 128)

In [ ]:
# with open('generated_queries_all.tsv', 'w') as fOut:
#     for start_idx in tqdm(range(0, len(paragraphs), batch_size)):
#         sub_paragraphs = paragraphs[start_idx:start_idx+batch_size]
#         inputs = tokenizer.prepare_seq2seq_batch(sub_paragraphs, max_length=max_length_paragraph, truncation=True, return_tensors='pt').to(device)
#         outputs = model.generate(
#             **inputs,
#             max_length=max_length_query,
#             do_sample=True,
#             top_p=0.95,
#             num_return_sequences=num_queries)

#         for idx, out in enumerate(outputs):
#             query = tokenizer.decode(out, skip_special_tokens=True)
#             query = _removeNonAscii(query)
#             para = sub_paragraphs[int(idx/num_queries)]
#             para = _removeNonAscii(para)
#             fOut.write("{}\t{}\n".format(query.replace("\t", " ").strip(), para.replace("\t", " ").strip()))

In [61]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models, datasets
from torch import nn
import os
import random

train_examples = [] 
with open('../input/user-query-data/generated_queries_all (1).tsv') as fIn:
    for line in fIn:
        try:
            query, paragraph = line.strip().split('\t', maxsplit=1)
            train_examples.append(InputExample(texts=[query, paragraph]))
        except:
            pass
        
random.shuffle(train_examples)

# For the MultipleNegativesRankingLoss, it is important
# that the batch does not contain duplicate entries, i.e.
# no two equal queries and no two equal paragraphs.
# To ensure this, we use a special data loader
train_dataloader = datasets.NoDuplicatesDataLoader(train_examples, batch_size=8)

# Now we create a SentenceTransformer model from scratch
word_emb = models.Transformer('sentence-transformers/msmarco-distilbert-base-dot-prod-v3')
pooling = models.Pooling(word_emb.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_emb, pooling])


# MultipleNegativesRankingLoss requires input pairs (query, relevant_passage)
# and trains the model so that is is suitable for semantic search
train_loss = losses.MultipleNegativesRankingLoss(model)


#Tune the model
num_epochs = 3
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=num_epochs, warmup_steps=warmup_steps, show_progress_bar=True)

os.makedirs('search', exist_ok=True)
model.save('search/search-model')

FileNotFoundError: ignored

In [ ]:
from sentence_transformers import SentenceTransformer, util
import gzip
import json
import os

model = SentenceTransformer('search/search-model')

In [ ]:
!zip -r search_model.zip "./search"

In [ ]:
import faiss
encoded_data = model.encode(df.Plot.tolist())
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(df))))
faiss.write_index(index, 'movie_plot.index')

In [ ]:
from pprint import pprint

query="Artificial Intelligence based action movie"
results=search(query, top_k=5, index=index, model=model)

print("\n")
for result in results:
    print('\t',result)

In [ ]:
from pprint import pprint

query="movie about romance and pain of seperation"
results=search(query, top_k=5, index=index, model=model)

print("\n")
for result in results:
    print('\t',result)